In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
noise_type = 'gaussian'      # 'gaussian', 'saltpepper', 'speckle'
model_type = 'conv'          # 'fc', 'conv'
epochs = 10
batch = 128
latent_dim = 64